In [ ]:
# import dependencies
import requests
import pandas as pd
import psycopg2
from datetime import datetime

# make an api request using url and Save the response content to a local file
response = requests.get("https://drive.google.com/file")
filename = "covid_19_data.csv"
if response.status_code == 200:
    with open(filename, 'wb') as file:
        file.write(response.content)
    print("CSV file downloaded successfully.")
else:
    print("Failed to download the CSV file.")

# change the Observation date column to datetime formate    
data = pd.read_csv("covid_19_data.csv")
date_string = data.ObservationDate
date_series = pd.to_datetime(date_string, format="%m/%d/%Y")
data.ObservationDate = date_series

# save the processed data as csv file to local machine
data.to_csv("newcovid_19_data.csv", index = False)

# define a cennection to postgress database
def get_db_connection(host, database, username, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=username,
        password=password
    )
    
    return conn

# initialize connection function parameters
host = "localhost"
database = "Covid19"
username = "postgres"
password = "******"

# establish a connect
conn = get_db_connection(host, database, username, password)
# create cursor
cur = conn.cursor()

# create table covid 19 data
create_statement = 'CREATE TABLE covid_19_data(Serial_number serial, Observation_Date date, Province_State VARCHAR(50), Country_Province VARCHAR(50), Last_Updated VARCHAR(50), Confirmed NUMERIC, Death NUMERIC, Recovered NUMERIC)'
cur.execute(create_statement)
conn.commit()

# load the data to the postgres database table
with open("newcovid_19_data.csv", "r") as f:
    next(f)
    cur.copy_from(f, "covid_19_data", sep=',')
    
# commit connection   
conn.commit()

#close cursor and connection
cur.close()
conn.close()